In [ ]:
#| default_exp route_provider


In [ ]:
#| export
from __future__ import annotations


# Routes provider

> Enables method-based routing in fasthtml by extending `APIRouter` capabilities.  

This allows objects to provide routes through their methods, similar to class-based views in web frameworks.


# Prologue

In [ ]:
#| export
import inspect
import typing
from types import MethodType
from typing import Any
from typing import Callable
from typing import ClassVar
from typing import Protocol

import fastcore.all as FC
from fasthtml.core import APIRouter
from fasthtml.core import FastHTML
from fasthtml.core import noop_body


In [ ]:
#| export
from bridget.helpers import id_gen
from bridget.helpers import nb_app


In [ ]:
from functools import partial
from typing import cast
from typing import Iterable
from typing import Type

import starlette.routing
import starlette.types
from fastcore.test import *
from fasthtml.components import Div
from fasthtml.core import uri
from rich.console import Console
from starlette.testclient import TestClient


----


In [ ]:
#| exporti

new_id = id_gen()


In [ ]:
cprint = (console := Console(width=120)).print


----

In [ ]:
def get_app():
    return (app := nb_app()), TestClient(app, base_url='http://nb', headers={'hx-request': '1'}), app.route

app, cli, rt = get_app()


# Methods as routes endpoints
> Understanding FastHTML's routing system and extending it to support instance methods.

:::{.callout-note}
In this docs I use widget/component interchangeably.
:::



### Why Method-Based Routes in Notebooks?

FastHTML typically encourages defining components in separate modules. While this works great for web applications, notebooks have different workflows:

**Interactive Development**  
Notebooks are all about exploration and rapid prototyping. Everything happens in cells - they're our unit of work. Notebooks are more akin to one page apps. When you're quickly testing ideas or building one-time components, creating separate modules can feel like unnecessary overhead. Method-based routes let us define and test components right where we need them, with scoping and encapsulation.

**State Management**  
When building widgets, we often need to maintain state. Instance methods make this natural. While both modules and classes can handle state effectively, sometimes one approach fits better than the other. It's nice to have options.

**Quick Iteration**  
The real advantage comes when you're iterating on a component. You can write code, test it, and modify it all in the same context. Make a change, run the cell, see the results. No jumping between files or reloading modules needed.

This approach may complement FastHTML's module-based components by providing another option for route management.

### FastHTML's Route Requirements

FastHTML's routing system builds on Starlette's, which accepts both functions and methods as endpoints (besides ASGI classes). The only requirement is being a [callable](https://github.com/encode/starlette/blob/427a8dcf357597df27b2509b1ac436caf7708300/starlette/routing.py#L208):

```python
endpoint: typing.Callable[..., typing.Any]
```

FastHTML follows FastAPI's style of using decorators for route definition. Route endpoints must be:
1. Callables with a `__name__` attribute (for route identification) [FastHTML.add_route](https://fasthtml.readthedocs.io/en/latest/api.html#fasthtml.core.FastHTML.add_route)
2. Have type annotations for all parameters (for request validation)

While this works for functions, it creates challenges with instance methods:


In [ ]:
class AClass:
    @rt('/give/me/{a}')
    def get(self, a:str): return f'{self!r} {a!r}'

cli.get('/give/me/b').text


/Users/vic/miniforge3/envs/bridget/lib/python3.12/site-packages/fasthtml/core.py:188: UserWarning: `self has no type annotation and is not a recognised special name, so is ignored.
  if arg!='resp': warn(f"`{arg} has no type annotation and is not a recognised special name, so is ignored.")


"None 'b'"

1. **Method Binding**: When using decorators at class level, methods are still unbound functions:
   - The decorator sees the function `get`, not the bound method
   - The `self` parameter won't be properly valued

2. **Type Annotations**: FastHTML expects all parameters to be annotated:
   - The implicit `self` parameter lacks annotation
   - This triggers warnings in FastHTML's validation


As a workaround you could use `partial` to bind methods after instance creation. As partials don't have a `__name__` attribute, we need to hack it manually:


In [ ]:
class AClass:
    def get(self, a:str): return f'{self!r} {a!r}'

a = AClass()
# Manually create bound method and set name
(f := partial(a.get.__func__, a)).__name__ = 'get'  # type: ignore
rt('/give/me/{a}')(f)

cli.get('/give/me/b').text


"<__main__.AClass object> 'b'"

However, this approach is verbose and error-prone, separates route definition from method implementation, and makes code harder to maintain, besides being cumbersome and ugly, clearly time for some more python dark magic here.  

We'll extend FastHTML's routing capabilities by:
1. Creating an enhanced `APIRouter` class that preserves method references
2. Supporting automatic method binding at instance creation
3. Enabling property-based routing for GET/POST/PUT/DELETE operations
4. Providing automatic route mounting and path generation

We aim for a more natural and maintainable syntax:

```python
class Widget:
    ...
    @ar('/value')
    def get_value(self): 
        return self._value
```

:::{.callout-note}
We'll use the handy `APIRouter` to define routes providers.  
`APIRouter` it's defined but not used anywhere (version 0.9.1). I'm not sure what's' Jeremy's intention for this class, but it seems appropiate using it here for our purpose.
:::


# APIRouterB
> Enhanced Router for Method-Based Routes

Extends FastHTML's APIRouter to preserve method references while enabling route registration.


In [ ]:
#| export

class APIRouterB(APIRouter):
    routes: list  # here to make type-checkers happy
    idx: str = '' # Instance identifier
    to: str = ''  # Base path for routes
    name: str = '' # Router name
    
    def __init__(self, routes=None, wss=None): 
        self.routes,self.wss = FC.listify(routes or [], use_list=True), FC.listify(wss or [], use_list=True)

    def __call__(self, path:str|None=None, methods=None, name=None, include_in_schema=True, body_wrap=noop_body):
        "Add a route at `path`"
        def f(func): 
            self.routes.append((func, path,methods,name,include_in_schema,body_wrap))
            return func
        return f(path) if callable(path) else f



## APIRouterB: Enhanced APIRouter for Method-Based Routes

For reasons unclear to me, APIRouter wipes out the functions when defining routes unlike normal `@rt` decorator.
We need to keep the functions around in order for the class to get its methods.

`APIRouterB` preserves the original method references while collecting routes. This allows methods to work both as routes and methods, with proper instance binding. Like FastHTML's standard APIRouter, routes are stored but not registered immediately, which lets us bind them to instances at mount time. This delayed registration enables stateful components and works naturally with Python properties and methods, maintaining compatibility with all FastHTML's routing features.


In [ ]:
class Counter:
    __ar__ = APIRouterB()
    
    def __init__(self): self._value = 0
    
    @__ar__('/value')
    def get_value(self): 
        "Get current counter value"
        return str(self._value)  # `0` is not a valid FastHTML response
    
    @__ar__('/inc')
    def increment(self): 
        "Increment counter and return new value"
        self._value += 1
        return self.get_value()

# Routes will work both as HTTP endpoints and methods
counter = Counter()
counter.increment()  # Works as method


'1'

We can't use the routes just yet, they aren't installed, APIRouter doesn't register them until we call `to_app`.


# RoutesProvider
> Any object with an `APIRouter` attribute can be a provider of routes.


In [ ]:
#| export

@typing.runtime_checkable
class RoutesProviderP(Protocol):
    __ar__: ClassVar[APIRouterB]
    ar: APIRouterB  # instance property with the final routes

class RoutesProvider:
    __ar__: ClassVar[APIRouterB]
    ar: APIRouterB
    def __init_subclass__(cls): 
        if not '__ar__' in vars(cls): setattr(cls, '__ar__', APIRouterB())
    # def __new__(cls, *args, **kwargs):
    #     if not hasattr(cls, '__ar__'): cls.__ar__ = APIRouterB()
    #     self = object.__new__(cls)
    #     return self

def _provider_routes(o: object):
    # Walk up the MRO chain, skipping object
    for base in (o if isinstance(o, type) else type(o)).__mro__[:-1]:  
        if not isinstance(base, type) or not hasattr(base, '__ar__'): continue
        yield from base.__ar__.routes
def provider_routes(prov: object):
    "Yield all route descriptors from class hierarchy in mro order"
    yield from _provider_routes(prov)



In [ ]:
def all_bases(cls: type, stop: type|None=None):
    "Return all superclasses of cls"
    bb = []
    for t in cls.__mro__[:-1]:  # exclude object
        if t is stop: break
        bb.append(t)
    return bb


class A(RoutesProvider):
    __ar__ = APIRouterB()
    @__ar__('/a')
    def a(self): return 'a'

class B(A):
    __ar__ = APIRouterB()
    @__ar__('/b')
    def b(self): return 'b'

class C(B):
    __ar__ = APIRouterB()
    @__ar__('/c?v={v}')
    def c(self, v:int=0): return f'c {v}'

class D(A):
    __ar__ = APIRouterB()
    @__ar__('/d')
    def d(self): return 'd'

class E(B, D):
    __ar__ = APIRouterB()
    @__ar__('/a')
    def e(self): return 'e'

test_eq(all_bases(C, RoutesProvider), [C, B, A])
test_eq(all_bases(E), [E, B, D, A, RoutesProvider])

test_eq([_[1] for _ in list(provider_routes(C))],  ['/c?v={v}', '/b', '/a'])
test_eq([_[1] for _ in list(provider_routes(E))],  ['/a', '/b', '/d', '/a'])


In [ ]:
app.routes.clear()

for f,p,m,n,i,bw in provider_routes(C):
    rt = starlette.routing.Route(p, f, methods=m, name=n, include_in_schema=i)
    app.add_route(rt)

routes = cast(list[starlette.routing.Route], app.routes)
test_eq([_.path for _ in routes], ['/c?v={v}', '/b', '/a'])

c_rt, *_ = routes

scope: starlette.types.Scope = {'type': 'http', 'path': '/c?v=3', 'method': 'GET'}
match, child_scope = c_rt.matches(scope)
test_eq(match, starlette.routing.Match.FULL)

scope: starlette.types.Scope = {'type': 'http', 'path': '/c?v=7', 'method': 'PUT'}
match, child_scope = c_rt.matches(scope)
test_eq(match, starlette.routing.Match.PARTIAL)


In [ ]:
app.routes.clear()

for f,p,m,n,i,bw in provider_routes(E):
    rt = starlette.routing.Route(p, f, methods=m, name=n, include_in_schema=i)
    app.add_route(rt)

routes = cast(list[starlette.routing.Route], app.routes)
test_eq([_.path for _ in routes], ['/a', '/b', '/d', '/a'])

e_rt, *_ = routes

scope: starlette.types.Scope = {'type': 'http', 'path': '/a', 'method': 'GET'}
match, child_scope = e_rt.matches(scope)
test_eq(match, starlette.routing.Match.FULL)
test_eq(child_scope['endpoint'], e_rt.endpoint)


`provider_routes` returns the routes in MRO order. We'll add the routes in that order, so Starlette will match them in that order too (NOTE: check this is documented). As such, more specific endpoints have precedence over more generic ones.

# Adding routes to the app
> Functions for registering routes from providers into FastHTML applications.


In [ ]:
#| export

def _ar_from_provider(prov:RoutesProviderP, name:str|None=None):
    ar, rr, funcs = (ar := APIRouterB()), ar.routes, []
    for f,_path,methods,_name,include_in_schema,body_wrap in provider_routes(prov):
        if inspect.isfunction(f):
            rr.append((getattr(prov, f.__name__), _path,methods,_name,include_in_schema,body_wrap))  # use method
            funcs.append(f)
        elif isinstance(f, property):
            for func,*m in ((f.fget, 'GET'), (f.fset, 'POST', 'PUT'), (f.fdel, 'DELETE')):
                if not func or func in funcs: continue
                mth = MethodType(func, prov)
                rr.append((mth, _path,methods or m,_name,include_in_schema,body_wrap))
                funcs.append(func)
    return ar


def add_routes(self: FastHTML, 
        prov:APIRouterB|RoutesProviderP,  # Router or provider containing routes
        mount:bool=False,                 # If True, mount routes under a sub-path
        path:str|None=None,               # Optional base path for mounting
        name:str|None=None                # Optional name for the route group
    ):
    "Register routes from a provider into a FastHTML app"
    if isinstance(prov, APIRouterB):
        is_ar, ar = True, prov
        if not mount and ar.routes: prov.to_app(self); return prov
    else:
        is_ar, ar = False, _ar_from_provider(prov, name)
        setattr(prov, 'ar', ar)
    ar.idx = new_id(prov)
    if not path: path = f"/{ar.idx}"; name = name or path.strip('/')
    ar.to, ar.name = path if mount else '', name or ''
    cls, rapp = type(prov), nb_app() if mount else self  # type: ignore
    # can't use ar.to_app(rapp), as we need to get the modified endpoints
    # to provide route introspection for methods
    for args in ar.routes:
        lf = rapp._add_route(*args)
        # a property can't have to()
        if not is_ar and not isinstance(getattr(cls, args[0].__name__), property):
            setattr(prov, args[0].__name__, lf)
    if mount and ar.routes: 
        self.mount(path, rapp, name=name)
    return ar


The `add_routes` function handles two scenarios:
1. Adding routes directly from an `APIRouterB` instance (if `mount` is `False`, this is equivalent to `to_app`)
2. Converting a `RoutesProvider` object's methods into routes

**Route Introspection**: Methods, like functions used in routes, get its .to() attribute because we use  `FastHTML._add_route` (caveat, private). Not properties, though, they're properties.


You can optionally mount the routes to obtain scoping. so routes can be organized at two levels:
1. **Notebook Level**: Global routes defined with `app`
2. **Provider Level**: Scoped routes defined by providers

This allows for clean organization of routes and natural encapsulation of component behavior directly in the notebook.

Though it's convenient, you don't need to mount routes providers, use its routes directly with the root level app.


In [ ]:
add_routes(app, counter)
cli.get('/inc').text


'2'

Add routes to notebook level app

In [ ]:
def mount(app: FastHTML, prov:APIRouterB|RoutesProviderP, path:str|None=None, name:str|None=None):
    return add_routes(app, prov, True, path, name)


mount(app, counter, '/counter')  # Routes under /counter/...
cli.get('/counter/value').text


'2'

Use `mount` to define routes providers with instances. In this way we can have notebook level routes, those defined with global app.route, and scoped routes defined with providers and add_routes.


Mount with explicit path


In [ ]:
mount(app, counter)

cli.get(f"/{counter.ar.idx}/inc").text


'3'

Mount routes with automatic path generation.

Routes available under `/{counter.ar.idx}/...`

More examples:

In [ ]:
class TestProvider:
    __ar__ = APIRouterB()
    ar = __ar__
    _value: int = 5

    @__ar__('/')
    @property
    def value(self): 
        return f'{self._value}'  # FastHTML hates zeroes :)
    
    @__ar__('/')
    @value.setter
    def value(self, x:int): 
        self._value = x
        return self.value
    
    @__ar__('/')
    @value.deleter
    def value(self): 
        try: del self._value
        except: pass
    
    @__ar__('/x/{y}', name='mul')
    def get(self, y:int): 
        return Div(link=uri(f"mul", y=f"{y}"))(self._value*y)


app, cli, rt = get_app()

prvr = TestProvider()
mount(app, prvr, '/prv', 'prv')
display(cli.get('/prv/').text)

test_eq(prvr.ar.to, '/prv')
test_eq(prvr.get.to(), '/x/{y}')
test_eq(app.url_path_for('prv:mul', y='3'), '/prv/x/3')
test_eq(app.url_path_for('prv:value'), '/prv/')

test_eq(cli.get('/prv/x/3').text, ' <div href="/prv/x/3">15</div>\n')
test_eq(cli.get('/prv/').text, '5')
test_eq(cli.post('/prv/', data={'x':'7'}).text, '7')
test_eq(cli.put('/prv/', data={'x':'3'}).text, '3')
test_eq(cli.get('/prv/x/3').text, ' <div href="/prv/x/3">9</div>\n')
test_eq(cli.delete('/prv/').text, '')
test_eq(cli.get('/prv/x/3').text, ' <div href="/prv/x/3">15</div>\n')

cli.get('/prv/x/3').text


'5'

' <div href="/prv/x/3">15</div>\n'

Note that we can also set properties as routes. Use as above, or simply apply once in the last descriptor function.

In [ ]:
class TestProvider:
    ar = __ar__ = APIRouterB()
    _value: int = 5

    @property
    def value(self): return f'{self._value}'  # FastHTML hates zeros

    @value.setter
    def value(self, x:int): 
        self._value = x
        return self.value
    
    @ar('/')
    @value.deleter
    def value(self): 
        try: del self._value
        except: pass
    
    @ar('/x/{y}')
    def get(self, y:int):
        return self._value*y


app, cli, rt = get_app()

prvr = TestProvider()
mount(app, prvr, '/prv', 'prv')
display(cli.get('/prv/').text)

test_eq(cli.get('/prv/x/3').text, '15')
test_eq(cli.get('/prv/').text, '5')
test_eq(cli.post('/prv/', data={'x':'7'}).text, '7')
test_eq(cli.put('/prv/', data={'x':'3'}).text, '3')
test_eq(cli.get('/prv/x/3').text, '9')
test_eq(cli.delete('/prv/').text, '')
test_eq(cli.get('/prv/x/3').text, '15')

cli.get('/prv/x/3').text


'5'

'15'

If we don't specify a path, a new one is generated with a unique id.

In [ ]:
class TestProvider:
    ar = __ar__ = APIRouterB()
    _value: int = 5

    @property
    def value(self): return f'{self._value}'  # FastHTML hates zeros

    @value.setter
    def value(self, x:int): 
        self._value = x
        return self.value
    
    @ar('/')
    @value.deleter
    def value(self): 
        try: del self._value
        except: pass
    
    @ar('/x/{y}')
    def get(self, y:int): return self._value*y


app, cli, rt = get_app()

prvr = TestProvider()
ar: APIRouterB = mount(app, prvr)  # type: ignore
p = ar.to
print(p)

test_eq(cli.get(f"{p}/x/3").text, '15')
test_eq(cli.get(f"{p}/").text, '5')
test_eq(cli.post(f"{p}/", data={'x':'7'}).text, '7')
test_eq(cli.put(f"{p}/", data={'x':'3'}).text, '3')
test_eq(cli.get(f"{p}/x/3").text, '9')
test_eq(cli.delete(f"{p}/").text, '')
test_eq(cli.get(f"{p}/x/3").text, '15')

test_eq(app.url_path_for(ar.name, path='/'), f'{p}/')

cli.get(f"{p}/x/3").text


/TestProvider_4823907552


'15'

# APIRoute
> Decorator to define route endpoints with methods.

We can simplyfy all that by using a decorator.


In [ ]:
#| export

class APIRoute:
    def __init__(self, path:str|None=None, methods=None, name=None, include_in_schema=True, body_wrap=noop_body):
        self.rargs = path, methods, name, include_in_schema, body_wrap

    def __call__(self, func: Callable[..., Any]) -> Callable: self.func = func; return self

    def __set_name__(self, owner, name):
        if not '__ar__' in vars(owner): setattr(owner, '__ar__', APIRouterB())
        owner.__ar__.routes.append((self.func, *self.rargs))
        setattr(owner, name, self.func)  # let methods be methods
    
ar = APIRoute


`APIRoute` a.k.a `ar` is a decorator to define route endpoints with methods. It's also a descriptor to populate lazily the `APIRouter` of the class.  
By using it, you're implicitly converting the class in a `RoutesProviderP`.


In [ ]:
class RProvider:
    a:int=0

    @ar('/a', name='changed')
    def changed(self, a:int, req):
        setattr(self, 'a', a)
        return f"{a}, {req.url_for('changed')}"


app, cli, rt = get_app()

rp = RProvider()
add_routes(app, rp)

test_eq(cli.post('/a', data={'a':'7'}).text, '7, http://nb/a')


If you, like me, love annotations but hates the stupid wiggly reds, inherit from `RoutesProvider`.

In the bridget notebook we'll see that we can also auto mount routes providers.

# Colophon
----


In [ ]:
import fastcore.all as FC
import nbdev
from nbdev.clean import nbdev_clean


In [ ]:
if FC.IN_NOTEBOOK:
    nb_path = '20_route_provider.ipynb'
    # nbdev_clean(nb_path)
    nbdev.nbdev_export(nb_path)
